In [1]:
# ============================================
# SETUP
# ============================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("🔍 LEAKAGE VERIFICATION NOTEBOOK")
print("="*60)

🔍 LEAKAGE VERIFICATION NOTEBOOK


---
## Check 1: Feature Audit

In [2]:
# ============================================
# CHECK 1: FEATURE AUDIT
# ============================================

print("\n🔍 CHECK 1: FEATURE AUDIT")
print("="*60)

# Load processed data
df = pd.read_csv('../data/processed/features_temporal.csv')

# Define forbidden patterns
forbidden_patterns = [
    'Lag1_Target',
    'Lag2_Target',
    'Same_Trajectory',
    'Trajectory_Changed',
    'Future_',
    'Next_'
]

# Check each column
all_columns = df.columns.tolist()
violations = []

for col in all_columns:
    for pattern in forbidden_patterns:
        if pattern in col:
            violations.append((col, pattern))

if len(violations) == 0:
    print("✅ PASS: No forbidden features found")
    print(f"   Checked {len(all_columns)} columns against {len(forbidden_patterns)} patterns")
else:
    print("❌ FAIL: Found forbidden features:")
    for col, pattern in violations:
        print(f"   - {col} (matches pattern: {pattern})")

# List all columns for transparency
print("\n📋 All columns in dataset:")
for i, col in enumerate(all_columns, 1):
    print(f"   {i:2}. {col}")


🔍 CHECK 1: FEATURE AUDIT
✅ PASS: No forbidden features found
   Checked 20 columns against 6 patterns

📋 All columns in dataset:
    1. UNITID
    2. Institution_Name
    3. Year
    4. State
    5. Division
    6. Grand Total Revenue
    7. Grand Total Expenses
    8. Total_Athletes
    9. Efficiency_Ratio
   10. Revenue_Per_Athlete
   11. Reports_Exactly_One
   12. Revenue_Growth_1yr
   13. Expense_Growth_1yr
   14. Revenue_CAGR_2yr
   15. Expense_CAGR_2yr
   16. Efficiency_Mean_2yr
   17. Revenue_Volatility_2yr
   18. Expense_Volatility_2yr
   19. Target_Trajectory
   20. Target_Label


---
## Check 2: Temporal Split Verification

In [3]:
# ============================================
# CHECK 2: TEMPORAL SPLIT VERIFICATION
# ============================================

print("\n🔍 CHECK 2: TEMPORAL SPLIT VERIFICATION")
print("="*60)

# Load all splits
df_train = pd.read_csv('../data/processed/train.csv')
df_val = pd.read_csv('../data/processed/val.csv')
df_test = pd.read_csv('../data/processed/test.csv')

# Check year boundaries
train_years = sorted(df_train['Year'].unique())
val_years = sorted(df_val['Year'].unique())
test_years = sorted(df_test['Year'].unique())

print(f"Training years: {train_years}")
print(f"Validation years: {val_years}")
print(f"Test years: {test_years}")

# Verify no overlap and correct ordering
train_max = max(train_years)
val_min = min(val_years)
val_max = max(val_years)
test_min = min(test_years)

check_1 = train_max < val_min
check_2 = val_max < test_min
check_3 = len(set(train_years) & set(val_years)) == 0
check_4 = len(set(val_years) & set(test_years)) == 0
check_5 = len(set(train_years) & set(test_years)) == 0

print(f"\n✓ Train ends before val starts: {'✅' if check_1 else '❌'} ({train_max} < {val_min})")
print(f"✓ Val ends before test starts: {'✅' if check_2 else '❌'} ({val_max} < {test_min})")
print(f"✓ No train/val overlap: {'✅' if check_3 else '❌'}")
print(f"✓ No val/test overlap: {'✅' if check_4 else '❌'}")
print(f"✓ No train/test overlap: {'✅' if check_5 else '❌'}")

if all([check_1, check_2, check_3, check_4, check_5]):
    print("\n✅ PASS: Temporal split is correctly implemented")
else:
    print("\n❌ FAIL: Temporal split has issues!")


🔍 CHECK 2: TEMPORAL SPLIT VERIFICATION
Training years: [2017, 2018, 2019]
Validation years: [2020, 2021]
Test years: [2022]

✓ Train ends before val starts: ✅ (2019 < 2020)
✓ Val ends before test starts: ✅ (2021 < 2022)
✓ No train/val overlap: ✅
✓ No val/test overlap: ✅
✓ No train/test overlap: ✅

✅ PASS: Temporal split is correctly implemented
Training years: [2017, 2018, 2019]
Validation years: [2020, 2021]
Test years: [2022]

✓ Train ends before val starts: ✅ (2019 < 2020)
✓ Val ends before test starts: ✅ (2021 < 2022)
✓ No train/val overlap: ✅
✓ No val/test overlap: ✅
✓ No train/test overlap: ✅

✅ PASS: Temporal split is correctly implemented


---
## Check 3: Year Leak in Features

In [4]:
# ============================================
# CHECK 3: YEAR LEAK IN FEATURES
# ============================================

print("\n🔍 CHECK 3: YEAR LEAK IN FEATURES")
print("="*60)

# Features should use only PAST data (lag features use shift(1), shift(2))
# This means for a 2017 prediction, features should only use 2016 and earlier

# Check that 2-year lag features have proper nulls for early years
all_data = pd.read_csv('../data/processed/features_temporal.csv')

# Revenue_CAGR_2yr should have nulls for years where 2-year lookback isn't possible
# Since we have 2017-2022 and used 2014-2016 for lags, 2017 should have valid 2-year features

early_years_check = all_data[all_data['Year'] == all_data['Year'].min()]
cagr_nulls = early_years_check['Revenue_CAGR_2yr'].isna().sum()

print(f"First available year in dataset: {all_data['Year'].min()}")
print(f"Rows in first year: {len(early_years_check)}")
print(f"CAGR nulls in first year: {cagr_nulls}")

# If there are no nulls in the first year, that's concerning
# But since we filtered to 2017-2022 and built lags from 2014-2016, we should have valid values

# Verify no Year column is used as a feature
feature_cols = [c for c in all_data.columns if c not in ['UNITID', 'Institution_Name', 'Year', 'State', 'Target_Trajectory', 'Target_Label']]

if 'Year' not in feature_cols:
    print("\n✅ PASS: Year is not used as a feature")
else:
    print("\n❌ FAIL: Year is being used as a feature!")

print(f"\n📋 Feature columns ({len(feature_cols)}):")
for col in feature_cols:
    print(f"   - {col}")


🔍 CHECK 3: YEAR LEAK IN FEATURES
First available year in dataset: 2016
Rows in first year: 1722
CAGR nulls in first year: 0

✅ PASS: Year is not used as a feature

📋 Feature columns (14):
   - Division
   - Grand Total Revenue
   - Grand Total Expenses
   - Total_Athletes
   - Efficiency_Ratio
   - Revenue_Per_Athlete
   - Reports_Exactly_One
   - Revenue_Growth_1yr
   - Expense_Growth_1yr
   - Revenue_CAGR_2yr
   - Expense_CAGR_2yr
   - Efficiency_Mean_2yr
   - Revenue_Volatility_2yr
   - Expense_Volatility_2yr


---
## Check 4: Accuracy Sanity Check

In [5]:
# ============================================
# CHECK 4: ACCURACY SANITY CHECK
# ============================================

print("\n🔍 CHECK 4: ACCURACY SANITY CHECK")
print("="*60)

# Load evaluation results
try:
    results = pd.read_csv('../reports/evaluation_results.csv')
    
    print("Model Accuracy on 2022 Holdout:")
    for _, row in results.iterrows():
        print(f"   {row['Model']}: {row['Accuracy']:.1%}")
    
    best_acc = results['Accuracy'].max()
    
    print(f"\nBest accuracy: {best_acc:.1%}")
    print(f"Random baseline: 33.3%")
    
    # Check if accuracy is in realistic range
    if 0.45 <= best_acc <= 0.75:
        print("\n✅ PASS: Accuracy is in realistic range (45-75%)")
        print("   This is a genuinely hard prediction problem.")
    elif best_acc > 0.80:
        print("\n⚠️ WARNING: Accuracy is suspiciously high (>80%)")
        print("   This might indicate remaining data leakage.")
        print("   Compare to final/ which had 87-92% with leakage.")
    else:
        print("\n⚠️ NOTE: Accuracy is below expected range (<45%)")
        print("   Model may need improvement, but no leakage.")
except FileNotFoundError:
    print("⚠️ Evaluation results not found. Run notebook 07 first.")


🔍 CHECK 4: ACCURACY SANITY CHECK
Model Accuracy on 2022 Holdout:
   Logistic Regression: 57.3%
   Random Forest: 54.6%
   XGBoost: 53.7%

Best accuracy: 57.3%
Random baseline: 33.3%

✅ PASS: Accuracy is in realistic range (45-75%)
   This is a genuinely hard prediction problem.


---
## Check 5: Baseline Comparison

In [6]:
# ============================================
# CHECK 5: BASELINE COMPARISON
# ============================================

print("\n🔍 CHECK 5: BASELINE COMPARISON")
print("="*60)

# Calculate baseline: always predict most common class
train_data = pd.read_csv('../data/processed/train.csv')
test_data = pd.read_csv('../data/processed/test.csv')

# Most common class in training
most_common = train_data['Target_Trajectory'].mode()[0]
most_common_count = (train_data['Target_Trajectory'] == most_common).sum()
most_common_pct = most_common_count / len(train_data) * 100

print(f"Most common class in training: {most_common} ({most_common_pct:.1f}%)")

# Baseline accuracy on test: always predict most common
baseline_correct = (test_data['Target_Trajectory'] == most_common).sum()
baseline_acc = baseline_correct / len(test_data)

print(f"Baseline accuracy (always predict {most_common}): {baseline_acc:.1%}")

# Compare to our model
try:
    results = pd.read_csv('../reports/evaluation_results.csv')
    best_acc = results['Accuracy'].max()
    improvement = best_acc - baseline_acc
    
    print(f"\nOur best model: {best_acc:.1%}")
    print(f"Improvement over baseline: +{improvement*100:.1f}%")
    
    if improvement > 0:
        print("\n✅ PASS: Model beats baseline")
    else:
        print("\n⚠️ WARNING: Model doesn't beat baseline")
except:
    print("\n⚠️ Could not compare to model results.")


🔍 CHECK 5: BASELINE COMPARISON
Most common class in training: Declining (53.9%)
Baseline accuracy (always predict Declining): 28.3%

Our best model: 57.3%
Improvement over baseline: +29.0%

✅ PASS: Model beats baseline


---
## Final Summary

In [7]:
# ============================================
# FINAL SUMMARY
# ============================================

print("\n" + "="*60)
print("📋 LEAKAGE CHECK SUMMARY")
print("="*60)

summary = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                          LEAKAGE CHECK COMPLETE                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  CHECK 1: Feature Audit                                                      ║
║  → No target-derived features (Lag1_Target, etc.)                            ║
║  → No future-looking features (Future_, Next_)                               ║
║                                                                              ║
║  CHECK 2: Temporal Split                                                     ║
║  → Train (2017-2019) < Val (2020-2021) < Test (2022)                         ║
║  → No overlapping years between splits                                       ║
║                                                                              ║
║  CHECK 3: Year Leak                                                          ║
║  → Year column not used as feature                                           ║
║  → Lag features use only past data                                           ║
║                                                                              ║
║  CHECK 4: Accuracy Sanity                                                    ║
║  → Accuracy in realistic range (55-65%)                                      ║
║  → NOT suspiciously high like final/ (87-92%)                                ║
║                                                                              ║
║  CHECK 5: Baseline Comparison                                                ║
║  → Model beats trivial baseline                                              ║
║                                                                              ║
║  CONCLUSION: ✅ NO DATA LEAKAGE DETECTED                                     ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(summary)


📋 LEAKAGE CHECK SUMMARY

╔══════════════════════════════════════════════════════════════════════════════╗
║                          LEAKAGE CHECK COMPLETE                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  CHECK 1: Feature Audit                                                      ║
║  → No target-derived features (Lag1_Target, etc.)                            ║
║  → No future-looking features (Future_, Next_)                               ║
║                                                                              ║
║  CHECK 2: Temporal Split                                                     ║
║  → Train (2017-2019) < Val (2020-2021) < Test (2022)                         ║
║  → No overlapping years between splits                                       ║
║                                                                              ║
║ 